In [18]:
import time
import random

import numpy as np
import matplotlib.pyplot as plt

In [19]:
import torch
import torch.nn as nn
import torch.optim as Adam

import gymnasium as gym

In [20]:
class NN(nn.Module):

    def __init__(self, n_states, n_actions):
        super().__init__()
        self.n_states = n_states
        self.n_actions = n_actions
        
        self.relu = nn.ReLU()
        
        self.fc1 = nn.Linear(self.n_states, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 256)
        
        self.fcv4 = nn.Linear(256, 128)
        self.fcv5 = nn.Linear(128, self.n_actions)

        self.fca4 = nn.Linear(256, 128)
        self.fca5 = nn.Linear(128, 1)

    def forward(self, states):
        x = self.relu(self.fc3(self.relu(self.fc2(self.relu(self.fc1(states))))))

        values = self.fcv5(self.relu(self.fcv4(x)))
        adv = self.fca5(self.relu(self.fca4(x)))
                                                                         
        return values + (adv - torch.mean(adv, dim = 1, keepdim = True))

In [21]:
class PrioritizedReplayBuffer:
    def __init__(self, size = 100000, steps = 5, alpha = 0.9, beta = 0.1, beta_increment = 0.001, ep = 1e-5):
        self.size = size
        self.steps = steps
        self.alpha = alpha
        self.beta = beta
        self.beta_increment = beta_increment
        self.episilon = ep

        self.priority = deque(size)
        self.buffer = deque(size)
        self.n_step_buffer = deque(steps)

    def add(states, actions, reward, next_state, done):
        item = (states, actions, reward, next_state, done)
        self.n_step_buffer.append(item)
        if len(self.n_step_buffer) < self.steps and not done:
            return 
        
        state, action = self.n_step_buffer[0][:2]
        R = 0
        for idx, (_, _, r, n_s, d) in enumerate(self.n_step_buffer):
            R += (self.gamma ** idx) * r
            if d or idx == self.steps - 1:
                next_state, done, steps = n_s, d, idx + 1
                break
        
        self.buffer.append((state, action, R, next_state, done, steps))
        self.priority.append(self.priority.max() if self.buffe else 1.0)

    def sample(self, batch_size):
        prios = self.priority
        prob = prios ** a
        prob /= prob/prob.sum()

        indices = np.random.choice(len(self.buffer), batch_size, p = prob)
        samples = [self.buffer[i] for i in indices]

        weights = (len(self.buffer) * probs[indices]) ** (-self.beta)
        weights /= weights.max()

        self.beta = np.min([1, self.beta(1 + self.beta_increment)])

        state, action, reward, next_state, done, steps = zip(*samples)
        return state, action, reward, next_state, done, steps, indices, weights

    def update_priorities(self, indices, td_error):
        for idx, td_error in zip(indices, td_error):
            self.priority[idx] = abs(td+error) + self.episilon

        
        

In [22]:
class Agent:

    def __init__(self, env ,n_states, n_actions, gamma):
        self.env = env
        self.n_states = env.observation_space.samples()
        self.n_actions = env.action_space.samples()
        self.gamma = gamma
        self.target_update_freq = 100
        self.batch_size = 64

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.policy_net = DQN(n_states, n_actions)
        self.target_net = DQN(n_states, n_actions)
        self.buffer = PrioritizedReplayBuffer()

        self.policy_net.to(self.device)
        self.target_net.to(self.device)

        self.optimizer = Adam(self.policy_net.parameters, lr = 1e-5)
        self.addMemory()

    def addMemory(self):
        sum_rewards = 0
        state = self.env.reset()[0]
        for _ in range(1000):

            action = self.env.action_space.sample()
            next_state, reward, term, trun, _ = self.env.step(action)
            done = int(term or trun)
            sum_rewards += 1
            self.buffer.add(state, action, reward, next_state, done)
            if done:
                state = self.env.reset()[0]
            else:
                state = next_state



    def train(self):
        
        if len(self.buffer) < self.batch_size:
            return

        states, actions, rewards, next_states, done, steps, indices, weights = self.buffer.sample(self.batch_size)
        states = torch.tensor(states).to(self.device)
        actions = torch.tensor(actions).unsqueeze(1).to(self.device)
        rewards = torch.tensor(rewards).unsqueeze(1).to(self.device)
        next_states = torch.tensor(next_states).to(self.device)
        done = torch.tensor(done).unsqueeze(1).to(self.device)
        steps = torch.tensor(steps).unsqueeze(1).to(self.device)
        weights = torch.tensor(weights).unsqueeze(1).to(self.device)

        qValue = self.policy_net(states).gather(1, actions)

        with torch.no_grad():
            next_actions = self.policy_net(next_states).argmax(1, keepdim = True)
            target_net = self.target_net(next_states).gather(1, next_actions)
            target_qValues = rewards + self.gamma ** steps) * target_net * (1 - done)
        
        td_errors = target_qValues - qValue
        
        loss = F.mse_loss(qValue, target_qValues)
        loss = (weights * loss).mean()
        self.loss_value += loss
        self.optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(self.policy_net.parameters(), max_norm=1.0)
        self.optimizer.step()

        new_priorities = td_errors.detach().abs().cpu().numpy().flatten()
        self.buffer.update_priorities(indices, new_priorities)

        self.steps += 1
        if self.steps % self.target_update_freq == 0:
            self.target_net.load_state_dict(self.policy_net.state_dict())
    
    
    def playingLoop(self, num_episodes = 500):
        self.num_episodes = num_episodes
        self.rewards_per_episode = []
        self.loss_per_episode = []

        for ep in range(num_episodes):
            self.loss_value = 0
            state = self.env.reset()[0]
            episode_reward = 0
            done = 0

            while not done:

                action = self.select_action(state)
                next_state, reward, term, trun, _ = self.env.step(action)
                done = int(term or trun)
                self.buffer.add(state, action, reward, next_state, done)

                self.train()

                state = next_state
                episode_reward += reward

            # print((1-self.episilon_decay) * self.episilon)   
            self.episilon = max(self.episilon_min, self.episilon_decay * self.episilon)
            self.rewards_per_episode.append(episode_reward)
            self.loss_per_episode.append(self.loss_value.item())

            if ep % 50 == 0:
                avg_reward = np.mean(self.rewards_per_episode[-10:])
                print(f"Episode {ep} | Avg Reward: {avg_reward:.2f} | Epsilon: {self.episilon:.2f}")


IndentationError: unexpected indent (2915116139.py, line 53)